# 5 Sklearn Mistakes That Silently Tell You Are a Rookie
## No error messages - that's what makes them subtle...
![](images/unsplash.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://unsplash.com/@santabarbara77?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Varvara Grabova</a>
        on 
        <a href='https://unsplash.com/s/photos/mistake?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Unsplash</a>
    </strong>
</figcaption>